In [1]:
import geopandas as gpd
import pandas as pd
import datetime
import shapely

In [2]:
import sys
sys.path.append('..')

In [3]:
import rsutils.s2_grid_utils
import create_datacube
import sqlite_db_utils

In [4]:
roi_filepaths = {
    'north': '../../nh_crop_calendar/crop_calendar/data/Ethiopia_Bounding_Boxs/Ethiopia_2019_North_Bounding_Box.shp',
    'east': '../../nh_crop_calendar/crop_calendar/data/Ethiopia_Bounding_Boxs/Ethiopia_2021_East_Bounding_Box.shp',
    'west': '../../nh_crop_calendar/crop_calendar/data/Ethiopia_Bounding_Boxs/Ethiopia_2021_West_Bounding_Box.shp',
}

In [5]:
combined_gdf = gpd.GeoDataFrame(
    pd.concat([gpd.read_file(fp).to_crs('epsg:4326') for fp in roi_filepaths.values()]),
    crs = 'epsg:4326',
)

In [6]:
s2grids_gdf = rsutils.s2_grid_utils.get_s2_grids_gdf(
    geojson_epsg_4326 = shapely.unary_union(
        combined_gdf['geometry']
    ).__geo_interface__
)

In [7]:
s2grids_gdf.loc[s2grids_gdf.intersects(gpd.read_file(roi_filepaths['north']).to_crs('epsg:4326')['geometry'][0]), 'bounding_box'] = 'north_2019'
s2grids_gdf.loc[s2grids_gdf.intersects(gpd.read_file(roi_filepaths['east']).to_crs('epsg:4326')['geometry'][0]), 'bounding_box'] = 'east_2021'
s2grids_gdf.loc[s2grids_gdf.intersects(gpd.read_file(roi_filepaths['west']).to_crs('epsg:4326')['geometry'][0]), 'bounding_box'] = 'west_2021'

In [8]:
s2grids_gdf['bounding_box'].value_counts()

bounding_box
east_2021     580
north_2019    351
west_2021     113
Name: count, dtype: int64

In [9]:
s2grids_gdf['roi'] = 's2grid=' + s2grids_gdf['id']
del s2grids_gdf['id']
s2grids_gdf

,geometry,bounding_box,roi
0,"POLYGON ((37.29507 11.75306, 37.34212 11.74591...",north_2019,s2grid=1644b54
1,"POLYGON ((37.33784 11.74656, 37.38486 11.73941...",north_2019,s2grid=1644b5c
2,"POLYGON ((37.38059 11.74006, 37.42758 11.7329,...",north_2019,s2grid=1644b64
3,"POLYGON ((37.38059 11.77803, 37.42758 11.77085...",north_2019,s2grid=1644b6c
4,"POLYGON ((37.42331 11.7715, 37.47027 11.76432,...",north_2019,s2grid=1644b74
...,...,...,...
1039,"POLYGON ((39.36005 8.17639, 39.40566 8.17112, ...",east_2021,s2grid=17b52bc
1040,"POLYGON ((39.40152 8.1716, 39.4471 8.16633, 39...",east_2021,s2grid=17b52c4
1041,"POLYGON ((39.40152 8.20686, 39.4471 8.20156, 3...",east_2021,s2grid=17b52cc
1042,"POLYGON ((39.40152 8.13637, 39.4471 8.13112, 3...",east_2021,s2grid=17b52ec


In [10]:
LOCAL_DATACUBE_CATALOG_DB_PATH = '../data/datacubes/catalog.db'

In [11]:
s2l2a_datacube_catalog_gdf = sqlite_db_utils.fetch_rows_from_db(
    database = LOCAL_DATACUBE_CATALOG_DB_PATH,
    table = 'sentinel-2-l2a',
    timestamp_cols = ['last_update', 'startdate', 'enddate']
)

In [12]:
bb_s2l2a_datacube_catalog_gdf = s2l2a_datacube_catalog_gdf[
    s2l2a_datacube_catalog_gdf['roi'].isin(s2grids_gdf['roi'])
]

In [13]:
bb_s2l2a_datacube_catalog_gdf['datacube_filepaths'] = bb_s2l2a_datacube_catalog_gdf['local_folderpath'] + '/datacube.npy'

/var/folders/3c/6mw9fk952gv0vj80_n9sfr7m0000gn/T/ipykernel_57414/1429310321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_s2l2a_datacube_catalog_gdf['datacube_filepaths'] = bb_s2l2a_datacube_catalog_gdf['local_folderpath'] + '/datacube.npy'


In [14]:
bb_s2l2a_datacube_catalog_gdf.to_csv('../data/ethiopia/model_deploy_boundingboxes_2019-2022.csv', index=False)

In [15]:
s2grids_gdf.to_file('../data/ethiopia/boundingboxes_s2grids.geojson')

In [ ]:
s2grids_gdf.to_file('../data/ethiopia/bb_s2grids.geojson')

In [ ]:
pd.DataFrame(
    s2grids_gdf[['id']].rename(columns={'id':'s2gridid'})
).to_csv('../data/ethiopia_bb_s2grids.csv', index=False)

In [ ]:
with open('../data/ethiopia/boundingboxes_s2grids.txt', 'w') as f:
    f.writelines('s2grid=' + s2grids_gdf['id'] + '\n')